<a href="https://colab.research.google.com/github/AmirJlr/RecSys/blob/master/pytorch-geometric/football_player_recommender_pyg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch

os.environ['TF_USE_LEGACY_KERAS'] = '1'

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q networkx
!pip install -q tensorflow-recommenders

2.6.0+cu124
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.utils import train_test_split_edges
from sklearn.preprocessing import MinMaxScaler

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Get The Dataset

https://www.kaggle.com/datasets/stefanoleone992/ea-sports-fc-24-complete-player-dataset

In [4]:
# Install dependencies as needed:
!pip install -q kagglehub[pandas-datasets]

In [5]:
import kagglehub
from kagglehub import KaggleDatasetAdapter


male_players_path = "male_players.csv"

df_male_players = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "stefanoleone992/ea-sports-fc-24-complete-player-dataset",
  male_players_path)



male_teams_path = "male_players.csv"

df_male_teams = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "stefanoleone992/ea-sports-fc-24-complete-player-dataset",
  male_teams_path)


/usr/local/lib/python3.11/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(
/usr/local/lib/python3.11/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


In [6]:
df_male_players.head(3)

,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3


In [7]:
df_male_players.columns

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'update_as_of', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential',
       ...
       'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'],
      dtype='object', length=109)

In [8]:
# Select relevant columns
columns_to_keep = [
    'player_id', 'short_name', 'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
    'club_name', 'club_position', 'nationality_name', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic'
]
df = df_male_players[columns_to_keep]
df.head(3)

,player_id,short_name,player_positions,overall,potential,value_eur,wage_eur,club_name,club_position,nationality_name,pace,shooting,passing,dribbling,defending,physic
0,231747,K. Mbappé,"ST, LW",91,94,181500000.0,230000.0,Paris Saint Germain,LW,France,97.0,90.0,80.0,92.0,36.0,78.0
1,239085,E. Haaland,ST,91,94,185000000.0,340000.0,Manchester City,ST,Norway,89.0,93.0,66.0,80.0,45.0,88.0
2,192985,K. De Bruyne,"CM, CAM",91,91,103000000.0,350000.0,Manchester City,SUB,Belgium,72.0,88.0,94.0,87.0,65.0,78.0


In [9]:
# --- Data Cleaning ---
# Drop players without a club (free agents) as they don't form links in our graph
df.dropna(subset=['club_name'], inplace=True)

<ipython-input-9-ad514b6b0432>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['club_name'], inplace=True)


In [10]:
# Fill missing stat values (e.g., goalkeepers have no 'pace') with the median of the column
for col in ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']:
    df[col].fillna(df[col].median(), inplace=True)


<ipython-input-10-7389ef0fdf74>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
<ipython-input-10-7389ef0fdf74>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(df[col].median(), inplace=True)


In [11]:
# Convert value and wage to numeric, handling missing values
df['value_eur'] = pd.to_numeric(df['value_eur'], errors='coerce').fillna(0)
df['wage_eur'] = pd.to_numeric(df['wage_eur'], errors='coerce').fillna(0)

<ipython-input-11-e1f531118088>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['value_eur'] = pd.to_numeric(df['value_eur'], errors='coerce').fillna(0)
<ipython-input-11-e1f531118088>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wage_eur'] = pd.to_numeric(df['wage_eur'], errors='coerce').fillna(0)


In [12]:
# Display the cleaned data
print(df.shape)
print(df.isnull().sum())

(178156, 16)
player_id           0
short_name          0
player_positions    0
overall             0
potential           0
value_eur           0
wage_eur            0
club_name           0
club_position       0
nationality_name    0
pace                0
shooting            0
passing             0
dribbling           0
defending           0
physic              0
dtype: int64


# Part 1: Graph-based Link Prediction with PyTorch Geometric (PyG)

We'll represent players and clubs as nodes in a heterogeneous graph. An edge exists if a player plays for a club. Our goal is to train a Graph Neural Network (GNN) to predict non-existent but plausible edges (i.e., new signings).

### 1.1 Data Preprocessing for PyG
We need to convert our pandas DataFrame into a graph format that PyG understands (torch_geometric.data.HeteroData).

In [13]:
df.head()

,player_id,short_name,player_positions,overall,potential,value_eur,wage_eur,club_name,club_position,nationality_name,pace,shooting,passing,dribbling,defending,physic
0,231747,K. Mbappé,"ST, LW",91,94,181500000.0,230000.0,Paris Saint Germain,LW,France,97.0,90.0,80.0,92.0,36.0,78.0
1,239085,E. Haaland,ST,91,94,185000000.0,340000.0,Manchester City,ST,Norway,89.0,93.0,66.0,80.0,45.0,88.0
2,192985,K. De Bruyne,"CM, CAM",91,91,103000000.0,350000.0,Manchester City,SUB,Belgium,72.0,88.0,94.0,87.0,65.0,78.0
3,158023,L. Messi,"CF, CAM",90,90,41000000.0,23000.0,Inter Miami,RF,Argentina,80.0,87.0,90.0,94.0,33.0,64.0
4,165153,K. Benzema,"CF, ST",90,90,51000000.0,95000.0,Al Ittihad,RS,France,79.0,88.0,83.0,87.0,39.0,78.0


In [14]:
import torch
from torch_geometric.data import HeteroData
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# --- Create Mappings for Players and Clubs ---
# We need unique integer IDs for each player and club

player_encoder = LabelEncoder()
club_encoder = LabelEncoder()

df['player_idx'] = player_encoder.fit_transform(df['player_id'])
df['club_idx'] = club_encoder.fit_transform(df['club_name'])

<ipython-input-14-94c3afca401e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_idx'] = player_encoder.fit_transform(df['player_id'])
<ipython-input-14-94c3afca401e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['club_idx'] = club_encoder.fit_transform(df['club_name'])


In [15]:
# --- Define Node Features ---
# For players, we'll use their stats.
player_features_cols = ['overall', 'potential', 'value_eur', 'wage_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']
scaler = MinMaxScaler()
player_features = scaler.fit_transform(df[player_features_cols])

In [16]:
player_features.shape

(178156, 10)

In [17]:
# For clubs, we can aggregate the stats of their players. This is a simple but effective approach.
club_features = df.groupby('club_idx')[player_features_cols].mean()
club_features = scaler.transform(club_features) # Use the same scaler as players

In [18]:
club_features.shape

(1082, 10)

In [19]:
# Convert to tensors
player_x = torch.tensor(player_features, dtype=torch.float)
club_x = torch.tensor(club_features, dtype=torch.float)

In [20]:
df.head()

,player_id,short_name,player_positions,overall,potential,value_eur,wage_eur,club_name,club_position,nationality_name,pace,shooting,passing,dribbling,defending,physic,player_idx,club_idx
0,231747,K. Mbappé,"ST, LW",91,94,181500000.0,230000.0,Paris Saint Germain,LW,France,97.0,90.0,80.0,92.0,36.0,78.0,26340,710
1,239085,E. Haaland,ST,91,94,185000000.0,340000.0,Manchester City,ST,Norway,89.0,93.0,66.0,80.0,45.0,88.0,31344,600
2,192985,K. De Bruyne,"CM, CAM",91,91,103000000.0,350000.0,Manchester City,SUB,Belgium,72.0,88.0,94.0,87.0,65.0,78.0,8667,600
3,158023,L. Messi,"CF, CAM",90,90,41000000.0,23000.0,Inter Miami,RF,Argentina,80.0,87.0,90.0,94.0,33.0,64.0,2540,502
4,165153,K. Benzema,"CF, ST",90,90,51000000.0,95000.0,Al Ittihad,RS,France,79.0,88.0,83.0,87.0,39.0,78.0,3518,41


In [21]:
# --- Define Edges ---
# Edges go from players to clubs. The shape is [2, num_edges].
player_indices = torch.tensor(df['player_idx'].values, dtype=torch.long)
club_indices = torch.tensor(df['club_idx'].values, dtype=torch.long)
edge_index = torch.stack([player_indices, club_indices], dim=0)

In [22]:
edge_index

tensor([[26340, 31344,  8667,  ..., 17936, 20269, 10814],
        [  710,   600,   600,  ...,  1047,   394,   344]])

In [23]:
# --- Create the HeteroData Object ---
import torch_geometric.transforms as T


data = HeteroData()
data['player'].x = player_x
data['club'].x = club_x
data['player', 'plays_for', 'club'].edge_index = edge_index

data = T.ToUndirected()(data)

print(data)

HeteroData(
  player={ x=[178156, 10] },
  club={ x=[1082, 10] },
  (player, plays_for, club)={ edge_index=[2, 178156] },
  (club, rev_plays_for, player)={ edge_index=[2, 178156] }
)


## 1.2 Creating Training and Validation Splits
For link prediction, we can't just split nodes. We need to split edges. PyG has a handy transform for this.

In [24]:
from torch_geometric.transforms import RandomLinkSplit

# This transform splits the edges into training, validation, and test sets.
# It also adds negative edges (edges that don't exist) for training the binary classifier.

transform = RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    is_undirected=False, # Our graph is directed: player -> club
    add_negative_train_samples=True,
    neg_sampling_ratio=2.0, # For each positive edge, create 2 negative ones
    edge_types=[('player', 'plays_for', 'club')],
    rev_edge_types=[('club', 'rev_plays_for', 'player')] # Need to define reverse edges for the model
)

train_data, val_data, test_data = transform(data)
print("Train Data:", train_data)
print("Validation Data:", val_data)
print("Test Data:", test_data)

Train Data: HeteroData(
  player={ x=[178156, 10] },
  club={ x=[1082, 10] },
  (player, plays_for, club)={
    edge_index=[2, 142526],
    edge_label=[427578],
    edge_label_index=[2, 427578],
  },
  (club, rev_plays_for, player)={ edge_index=[2, 142526] }
)
Validation Data: HeteroData(
  player={ x=[178156, 10] },
  club={ x=[1082, 10] },
  (player, plays_for, club)={
    edge_index=[2, 142526],
    edge_label=[53445],
    edge_label_index=[2, 53445],
  },
  (club, rev_plays_for, player)={ edge_index=[2, 142526] }
)
Test Data: HeteroData(
  player={ x=[178156, 10] },
  club={ x=[1082, 10] },
  (player, plays_for, club)={
    edge_index=[2, 160341],
    edge_label=[53445],
    edge_label_index=[2, 53445],
  },
  (club, rev_plays_for, player)={ edge_index=[2, 160341] }
)


In [25]:
train_data.metadata()

(['player', 'club'],
 [('player', 'plays_for', 'club'), ('club', 'rev_plays_for', 'player')])

## 1.3 Building the GNN Model
We'll build a model using SAGEConv (GraphSAGE), which is great for heterogeneous graphs. The model will learn embeddings for players and clubs, and then a simple decoder will predict the link probability.

In [26]:
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero

class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # A simple MLP to predict the edge
        self.lin1 = torch.nn.Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        # Concatenate the embeddings of the source and destination nodes
        z = torch.cat([z_dict['player'][row], z_dict['club'][col]], dim=-1)
        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Encoder for each node type
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        # Make the encoder heterogeneous
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

model = Model(hidden_channels=64)

## 1.4 Training the GNN
We'll use Binary Cross-Entropy with Logits as our loss, as this is a binary classification problem (is there a link or not?).

In [27]:
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

train_data, val_data, test_data = train_data.to(device), val_data.to(device), test_data.to(device)

def train():
    model.train()
    optimizer.zero_grad()

    # Forward pass through the model
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['player', 'club'].edge_label_index)

    # Get the ground truth labels
    target = train_data['player', 'club'].edge_label

    loss = criterion(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['player', 'club'].edge_label_index)
    pred = pred.sigmoid() # Apply sigmoid to get probabilities
    target = data['player', 'club'].edge_label
    return roc_auc_score(target.cpu().numpy(), pred.cpu().numpy())


for epoch in tqdm(range(1, 301)):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)

    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}, Test AUC: {test_auc:.4f}')



  0%|          | 0/300 [00:00<?, ?it/s]

Epoch: 010, Loss: 0.3615, Val AUC: 0.8110, Test AUC: 0.8131
Epoch: 020, Loss: 0.3362, Val AUC: 0.8123, Test AUC: 0.8138
Epoch: 030, Loss: 0.3341, Val AUC: 0.8141, Test AUC: 0.8157
Epoch: 040, Loss: 0.3330, Val AUC: 0.8164, Test AUC: 0.8181
Epoch: 050, Loss: 0.3320, Val AUC: 0.8172, Test AUC: 0.8189
Epoch: 060, Loss: 0.3312, Val AUC: 0.8194, Test AUC: 0.8212
Epoch: 070, Loss: 0.3296, Val AUC: 0.8244, Test AUC: 0.8265
Epoch: 080, Loss: 0.3249, Val AUC: 0.8344, Test AUC: 0.8374
Epoch: 090, Loss: 0.3112, Val AUC: 0.8468, Test AUC: 0.8503
Epoch: 100, Loss: 0.2957, Val AUC: 0.8615, Test AUC: 0.8663
Epoch: 110, Loss: 0.2503, Val AUC: 0.8666, Test AUC: 0.8717
Epoch: 120, Loss: 0.2303, Val AUC: 0.8744, Test AUC: 0.8804
Epoch: 130, Loss: 0.2193, Val AUC: 0.8770, Test AUC: 0.8829
Epoch: 140, Loss: 0.2176, Val AUC: 0.8782, Test AUC: 0.8840
Epoch: 150, Loss: 0.2159, Val AUC: 0.8791, Test AUC: 0.8848
Epoch: 160, Loss: 0.2149, Val AUC: 0.8794, Test AUC: 0.8851
Epoch: 170, Loss: 0.2141, Val AUC: 0.879

## 1.5 Making Recommendations with the GNN
We'll get the final embeddings for all players and clubs, then find the players with the highest predicted link probability for a given club.

In [28]:
# Get final embeddings after training

model.eval()
with torch.no_grad():
    # Move the data object to the same device as the model
    data = data.to(device)
    final_embeddings = model.encoder(data.x_dict, data.edge_index_dict)

player_embs = final_embeddings['player']
club_embs = final_embeddings['club']

In [29]:
final_embeddings

{'player': tensor([[-0.5757, -0.7459,  0.5185,  ...,  0.5774,  0.8656, -0.9822],
         [-0.3929, -0.4936,  0.2633,  ...,  0.4092,  0.6332, -0.6615],
         [-0.3324, -0.3714,  0.1177,  ...,  0.3156,  0.5500, -0.5117],
         ...,
         [ 1.9209,  2.4021, -2.5805,  ..., -1.7466, -0.9938,  2.2994],
         [ 1.9175,  2.3886, -2.5456,  ..., -1.7531, -1.0091,  2.2832],
         [ 1.8696,  2.4250, -2.5527,  ..., -1.7843, -0.9083,  2.3102]],
        device='cuda:0'),
 'club': tensor([[ 0.3316, -0.4052, -0.1543,  ..., -0.1318, -0.0893,  0.3859],
         [ 0.3287, -0.3989, -0.1814,  ..., -0.1563, -0.0932,  0.4787],
         [ 0.3278, -0.3723, -0.1468,  ..., -0.1474, -0.0853,  0.4132],
         ...,
         [ 0.3302, -0.4046, -0.1566,  ..., -0.1428, -0.0650,  0.3896],
         [ 0.2000, -0.2343,  0.0221,  ..., -0.3591, -0.3458,  0.6801],
         [ 0.2928, -0.3680, -0.1165,  ..., -0.2106, -0.1731,  0.5218]],
        device='cuda:0')}

In [30]:
def recommend_players_for_club(club_name, top_k=5):
    # 1. Get the club's integer index
    club_idx = club_encoder.transform([club_name])[0]

    # 2. Get the club's embedding
    club_embedding = club_embs[club_idx]

    # 3. Find players currently at the club to exclude them
    current_player_indices = df[df['club_name'] == club_name]['player_idx'].values

    # 4. Calculate scores for all players
    all_player_indices = torch.arange(len(player_embs))

    # Create edge indices for all possible player-club links
    club_repeated_indices = torch.full_like(all_player_indices, fill_value=club_idx)
    all_possible_edge_indices = torch.stack([all_player_indices, club_repeated_indices], dim=0)

    # Get scores from the decoder
    with torch.no_grad():
        scores = model.decoder(final_embeddings, all_possible_edge_indices)
        scores = scores.sigmoid() # Convert to probabilities

    # 5. Mask out players already at the club
    scores[current_player_indices] = -1.0 # Set their score to a very low value

    # 6. Get top_k recommendations
    top_k_indices = torch.topk(scores, k=top_k).indices
    top_k_player_ids = player_encoder.inverse_transform(top_k_indices.cpu().numpy())

    # Retrieve player details from the original dataframe
    recs_df = df[df['player_id'].isin(top_k_player_ids)][['short_name', 'overall', 'player_positions', 'value_eur']].drop_duplicates(subset=['short_name'])

    print(f"Top {top_k} player recommendations for {club_name}:")
    print(recs_df)


In [31]:
# Let's get recommendations for a few clubs
recommend_players_for_club('Manchester City', top_k=5)

Top 5 player recommendations for Manchester City:
          short_name  overall player_positions   value_eur
4         K. Benzema       90           CF, ST  51000000.0
195    Marco Asensio       83               RW  38500000.0
1847        A. Lunin       74               GK   7000000.0
12970      Cañizares       62               GK    850000.0
18674        G. Bale       81           RW, RM  20500000.0


In [32]:
recommend_players_for_club('FC Barcelona', top_k=5)

Top 5 player recommendations for FC Barcelona:
            short_name  overall player_positions   value_eur
178    Nacho Fernández       83       CB, LB, RB  17500000.0
8392             Peter       66       RW, LW, LB   2100000.0
11822             Noel       63           ST, RW   1300000.0
11851           Marvel       63               CB   1200000.0
13837         T. Nakai       61          CAM, CF    850000.0
